<a href="https://colab.research.google.com/github/ValentinaEmili/Ethnicity-recognition/blob/main/majority_voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score

def merged_results(path):
  cnn_pred = pd.read_csv(f"{path}/CNN_predictions.csv")
  lightGBM_pred = pd.read_csv(f"{path}/LightGBM_predictions.csv")
  alexnet_pred = pd.read_csv(f"{path}/alexnet_predictions.csv")
  svm_pred = pd.read_csv(f"{path}/svm_predictions.csv")
  y_true = pd.read_csv(f"{path}/true_values.csv", index_col=False)

  cnn_pred["Image_Name"] = cnn_pred["Image_Name"].str.replace(".jpg", "", regex=False)
  lightGBM_pred["Image_Name"] = lightGBM_pred["Image_Name"].str.replace(".jpg", "", regex=False)
  alexnet_pred["Image_Name"] = alexnet_pred["Image_Name"].str.replace(".jpg", "", regex=False)
  svm_pred["Image_Name"] = svm_pred["Image_Name"].str.replace(".npy", "", regex=False)
  y_true["Image_Name"] = y_true["Image_Name"].str.replace(".npy", "", regex=False)

  cnn_pred = cnn_pred.set_index("Image_Name").sort_index()
  lightGBM_pred = lightGBM_pred.set_index("Image_Name").sort_index()
  alexnet_pred = alexnet_pred.set_index("Image_Name").sort_index()
  svm_pred = svm_pred.set_index("Image_Name").sort_index()
  y_true = y_true.set_index("Image_Name").sort_index()

  cnn_pred.rename(columns={"Predicted_Class": "Predicted_class_cnn"}, inplace=True)
  lightGBM_pred.rename(columns={"Predicted_Label": "Predicted_class_lightGBM"}, inplace=True)
  alexnet_pred.rename(columns={"Predicted_Class": "Predicted_class_alexnet"}, inplace=True)
  svm_pred.rename(columns={"Predicted_class": "Predicted_class_svm"}, inplace=True)

  merged = pd.concat([y_true, cnn_pred["Predicted_class_cnn"], lightGBM_pred["Predicted_class_lightGBM"], alexnet_pred["Predicted_class_alexnet"], svm_pred["Predicted_class_svm"]], axis=1)
  return merged

In [6]:
def get_final_prediction(row):
  counter = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
  counter[row["Predicted_class_cnn"]] += 1
  counter[row["Predicted_class_lightGBM"]] += 1
  counter[row["Predicted_class_alexnet"]] += 1
  counter[row["Predicted_class_svm"]] += 1
  return int(max(counter, key=counter.get))

In [7]:
def main():
  features = ["eyes", "mouth", "nose"]
  for feature in features:
    folder_path = f"/content/{feature}"
    merged = merged_results(folder_path)
    merged["Final_prediction"] = merged.apply(get_final_prediction, axis=1)
    print(f"feature: {feature}")
    print(accuracy_score(merged["True_class"], merged["Final_prediction"]))
    print(classification_report(merged["True_class"], merged["Final_prediction"]))

if __name__ == "__main__":
  main()

feature: eyes
0.7340871021775545
              precision    recall  f1-score   support

           0       0.71      0.90      0.79      2018
           1       0.78      0.77      0.77       927
           2       0.87      0.67      0.76       670
           3       0.73      0.57      0.64       802
           4       0.44      0.21      0.28       359

    accuracy                           0.73      4776
   macro avg       0.71      0.62      0.65      4776
weighted avg       0.73      0.73      0.72      4776

feature: mouth
0.6656197654941374
              precision    recall  f1-score   support

           0       0.64      0.88      0.74      2018
           1       0.76      0.81      0.78       927
           2       0.60      0.36      0.45       670
           3       0.67      0.46      0.55       802
           4       0.51      0.09      0.15       359

    accuracy                           0.67      4776
   macro avg       0.64      0.52      0.54      4776
weighted a